In [1]:
import tkinter as tk
from tkinter.ttk import *
import webbrowser
import pandas as pd
import os

from tkinter import * 
import tkinter.messagebox
from tkinter import ttk
import tkinterweb
from tkinterhtml import HtmlFrame
import tkinterweb
from IPython.display import Audio
from urllib.request import urlopen

In [2]:
#pip install spotipy

In [3]:
#pip install pydub

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

cid = '3c72f1d18da74f63addd8423fd7d668f'
secret = '447a83ea7f494680a9bf88a43db64548'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
#sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

#sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(
token = util.prompt_for_user_token(
    username='12156878652',
    client_id = cid,
    client_secret = secret,
    redirect_uri = 'http://localhost:8000',
    scope=['user-modify-playback-state','user-read-playback-state']
    #'user-modify-playback-state'
    #['playlist-modify-public','user-read-currently-playing','user-read-playback-state']
    )
if token:
    sp = spotipy.Spotify(auth=token)

In [5]:
## Generate a prototype playlist for GUI development.
## Cody J Middleton

def generate_prototype_playlist():
    """
    Return a prototype playlist in the form of a pandas dataframe
    Spotify ID, Track Name, and Artist Name
    """
    # step 1 resolve path 
    temp_path = os.getcwd()
    root_path = temp_path.split('/napster_2')[0]
    repo_path = '/napster_2/lyric_genius_api/practice_data.csv'
    practice_data_path = root_path + repo_path

    # step 2 pull 30 random tracks.
    track_df = pd.read_csv(practice_data_path)
    rel_columns = ['track_id', 'track_name_x', 'artist_name_x']
    track_df = track_df[rel_columns]
    track_df.rename(columns={'track_id': 'spotify_id',
                            'track_name_x': 'track_name',
                            'artist_name_x': 'artist_name'
    })
    
    return track_df.sample(30, random_state=15)

In [6]:
track_df = generate_prototype_playlist()

In [7]:
track_df.head()

,track_id,track_name_x,artist_name_x
6242,5jzEwSyyymBlf1fa1o39T2,512,Mora
6876,2SJZdZ5DLtlRosJ2xHJJJa,Chance,Paulo Londra
6089,0joQFwiBKmi6X1Iq5R1ubH,Codeine Crazy,Future
4545,6sNBVMxGN6D9t4b9e2fnEN,Ruta 66,Pappo
449,3bmYpOfK6nLgr4GAtujhB8,Trenches,Pop Evil


In [8]:
track_df.shape

(30, 3)

In [61]:
#Action for clicking Get Lyric button
def reset():  
    global track_df
    global track_to_rate
    global like_dislike_counter
    
    sample_song_label.config(text="")
    #Once 10 songs have been rated, disable most buttons and allow user to view playlist
    if like_dislike_counter == 10:
        label.config(text="Track: \nArtist: ") #, font=("Arial",12))
        b_like["state"] = DISABLED
        b_dislike["state"] = DISABLED
        b_neutral["state"] = DISABLED
        b_listen["state"] = DISABLED
        b_playlist["state"] = NORMAL
    else:
        #display a random track for now
        track_to_rate = track_df.sample(n=1,replace=True) 
        #track_to_rate = track_df.iloc[like_dislike_counter] 
        label.config(text = 'Track: ' + track_to_rate.track_name_x.astype('str').item() + '\n' + 'Artist: ' + track_to_rate.artist_name_x.astype('str').item())

        if b_like["state"] == DISABLED:
            b_like["state"] = NORMAL
        if b_dislike["state"] == DISABLED:
            b_dislike["state"] = NORMAL
        if b_neutral["state"] == DISABLED:
            b_neutral["state"] = NORMAL
        if b_listen["state"] == DISABLED:
            b_listen["state"] = NORMAL
        if b_startover["state"] == DISABLED:
            b_startover["state"] = NORMAL
        #view_likes_dislikes()
    
# Add 1 to the rating counter and display progress to total songs needed to rate
def like_dislike_count():
    global like_dislike_counter
    like_dislike_counter += 1
    tracker_label = Label(frame1, text = 'Tracks Rated: ' + str("{0:0=2d}".format(like_dislike_counter)) + "/10")
    tracker_label.grid(row=2, column=0, sticky="nw", padx=5, pady = 10)
  
# Old version of tracking likes and dislikes
'''
def view_likes_dislikes():
    global data
    global root
    
    if not data:
        total_rows = 0
        total_columns = 1
    else:
        total_rows = len(data)
        total_columns = 1 #len(data[0])

    for i in range(total_rows):
        if data[i][2] == "Like":
            e = Entry(frame2, width=30, fg='green')
            e.grid(row=i, column=1, padx=5, pady=5)
            e.insert(END, data[i][1])
        else:
            e = Entry(frame2, width=30, fg='red')
            e.grid(row=i, column=1, padx=5, pady=5)
            e.insert(END, data[i][1])
'''
 
# Action for clicking like button: add to table at bottom of frame
def like():  
    global like_dislike_counter
    global listBox_lyrics
    global track_to_rate
    like_dislike_count()
    listBox_lyrics.insert("", "end", values=(track_to_rate.track_name_x.astype('str').item(), track_to_rate.artist_name_x.astype('str').item(), "Like")) 
    reset()
        
    
# Action for clicking Dislike button: add to table at bottom of frame  
def dislike():  
    global like_dislike_counter
    global listBox_lyrics
    global track_to_rate
    like_dislike_count()
    listBox_lyrics.insert("", "end", values=(track_to_rate.track_name_x.astype('str').item(), track_to_rate.artist_name_x.astype('str').item(), "Dislike")) 
    reset()
    
# Action for clicking Neutral button: add to table at bottom of frame  
def neutral():  
    global like_dislike_counter
    global listBox_lyrics
    global track_to_rate
    like_dislike_count()
    listBox_lyrics.insert("", "end", values=(track_to_rate.track_name_x.astype('str').item(), track_to_rate.artist_name_x.astype('str').item(), "Neutral")) 
    reset()
    
def listen_to_track():   
    global track_to_rate
    global sample_song_label
    track = track_to_rate.track_id.astype('str').item()
    track = sp.track(track)
    #print(track["preview_url"])
    
    try:
        #webbrowser.open_new(r"https://www.python.org")
        webbrowser.open_new(track["preview_url"])
        sample_song_label.config(text="Song preview playing in new window")
    except:
        sample_song_label.config(text="Song preview unavailable")
    
    '''
    frame_listen = tkinterweb.HtmlFrame(frame1)
    frame_listen.load_website(track["preview_url"]) 
    print(track["preview_url"])
    frame_listen.grid(row=1, column=1, pady=5, columnspan=3)
    '''
    #Audio(url="https://www.youtube.com/watch?v=5UMCrq-bBCg", autoplay=True)
 
# If you double click on a song in the display that was already rated, swap like / dislike
def enable_edit(event):
    curItem = listBox_lyrics.focus()
    curItem_values = listBox_lyrics.item(curItem)['values']
    if curItem_values[2] == 'Like':
        new_rating = 'Neutral'
    elif curItem_values[2] == 'Neutral':
        new_rating = 'Dislike'
    else:
        new_rating = 'Like'
    listBox_lyrics.item(curItem, values=(curItem_values[0], curItem_values[1],new_rating))
    for i in listBox_lyrics.selection():
        listBox_lyrics.selection_remove(i)
    
# Once enough songs are rated, the view playlist option displays and displays at bottom of frame
def view_playlist():
    # create table to show lyrics with rating
    global listBox_playlist
    global vsb_playlist
    
    # Hide lyric table
    #listBox_lyrics.grid_forget()
    
    vsb_playlist.grid(row=1, column=1, sticky='ns')
    listBox_playlist.grid(row=1, column=0, sticky="ew")
    
    #vsb.pack(side="right", fill="y") 
    #listBox_playlist.pack(side="left", fill="x", padx=5, pady=5)
    listBox_playlist.configure(yscrollcommand=vsb_playlist.set)
    
    #Insert songs from playlist dataframe
    for index, row in track_df.iterrows():
        listBox_playlist.insert("", "end", values=(row.track_name_x, row.artist_name_x))
    
    b_playlist["state"] = DISABLED
    b_export_playlist["state"] = NORMAL
    
# Reset button 
def start_over():
    global listBox_lyrics
    global listBox_playlist
    global like_dislike_counter
    
    # set counter back to 0 
    like_dislike_counter  = -1
    like_dislike_count()
    
    # Reset buttons
    b_playlist["state"] = DISABLED
    b_startover["state"] = DISABLED
    b_export_playlist["state"] = DISABLED
    
    # Remove lyric
    label.config(text="")
    
    # Reset table that displays playlist
    for item in listBox_playlist.get_children():
        listBox_playlist.delete(item)
    
    # Hide playlist and scrollbar
    listBox_playlist.grid_forget()
    vsb_playlist.grid_forget()
    
    # Reset table that displays liked / disliked lyrics
    for item in listBox_lyrics.get_children():
        listBox_lyrics.delete(item)
    
    listBox_lyrics.grid(row=1, column=0, sticky="ew")
    #listBox_lyrics.pack(side="left", fill=BOTH, padx=5, pady=5)
    lyricSearch["state"] = NORMAL
    reset()
    
def export_to_spotify():
    global sample_song_label
    for index, row in track_df.iterrows():
        try:
            sp.add_to_queue(row.track_id)
            sample_song_label.config(text="Songs have been added to your queue!")
        except:
            sample_song_label.config(text="Unable to add to playlist. Note: Spotify Premium required")
            break
    #sp.add_to_queue("13rC4iKtfQocWIfzPOJxaT")
    b_export_playlist["state"] = DISABLED

#Action for submitting lyric in search box
def lyric_processing():
    lyricText=lyricBox.get("1.0","end-1c")
    lyricSearch["state"] = DISABLED
    print(lyricText)     
    reset()
        

In [62]:
# global variables
# start with random lyric
track_to_rate = track_df.sample(n=1,replace=True) #.track_id.astype('str').item()
like_dislike_counter = 0

# song lyric display
# creating tkinter window
root = Tk()
root.geometry('625x450')
root.title("Napster 2")

frame0 = Frame(root)
frame1 = Frame(root)
frame2 = Frame(root)
frame3 = Frame(root)

# Frame for get lyric, like, dislike
frame0.grid(row=0, column=0, sticky="w")
frame0.grid_columnconfigure((0,1,2,3), weight=1) #, uniform="column")

# Frame for get lyric, like, dislike
frame1.grid(row=1, column=0, sticky="w")
frame1.grid_columnconfigure((0,1,2,3), weight=1, uniform="column")

#Add buttons to sub-frame in Frame 1 so they all fit in first column
frame1b = Frame(frame1)
frame1b.grid(row=1, column=0, sticky="ew")
frame1b.grid_columnconfigure((0,1,2,3), weight=1, uniform="column")

# Frame to display like / dislike selections and playlist
frame2.grid(row=2, column=0, sticky="ew", rowspan=2) #, padx=3, pady=5)

# Display buttons to view playlist, start over, export playlist to spotify
frame3.grid(row=5, column=0, sticky="ew")
#frame3.grid_columnconfigure((0,1,2,3), weight=1, uniform="column")
#frame3.grid_columnconfigure((0,1,2,3), weight=1, uniform="column")


# Add label to be used for displaying lyric
#label = Label(frame1, text = 'Track: ' + track_to_rate.track_name_x.astype('str').item() + '\n' + 'Artist: ' + track_to_rate.artist_name_x.astype('str').item(), anchor='w', justify=LEFT) #, font=("Arial",12))
label = Label(frame1, text = 'Track: ' + '\n' + 'Artist: ' , anchor='w', justify=LEFT) #, font=("Arial",12))
label.grid(row=0, column=0, sticky="nw", padx=3, pady = 10, columnspan=4)

# Add images to buttons
temp_path = os.getcwd()
root_path = temp_path.split('/napster_2')[0]
tu_repo_path = '/napster_2/user_interface/thumbs_up.png'
td_repo_path = '/napster_2/user_interface/thumbs_down.png'
r_repo_path = '/napster_2/user_interface/refresh.png'
p_repo_path = '/napster_2/user_interface/play_button.png'
logo_repo_path = '/napster_2/user_interface/napster_logo.png'

tu_path = root_path + tu_repo_path
td_path = root_path + td_repo_path
r_path = root_path + r_repo_path
p_path = root_path + p_repo_path
logo_path = root_path + logo_repo_path

# Creating a photoimage object to use image
tu_photo = PhotoImage(file = tu_path)
td_photo = PhotoImage(file = td_path)
r_photo = PhotoImage(file = r_path)
p_photo = PhotoImage(file = p_path)
logo_photo = PhotoImage(file = logo_path)

# Resizing image to fit on button
tu_photoimage = tu_photo.subsample(6, 6)
td_photoimage = td_photo.subsample(6, 6)
r_photoimage = r_photo.subsample(25, 25)
p_photoimage = p_photo.subsample(12, 12)
logo_photoimage = logo_photo.subsample(2, 2)

#Add logo to top frame
logo_label = Label(frame0, image = logo_photoimage)
logo_label.grid(row=0, column=0)

#Add search bar
lyricBox = Text(frame0, height=2, width=32)
lyricBox.grid(row=0, column=1, pady=5)

lyricSearch = Button(frame0, text = 'Submit', command=lyric_processing)
lyricSearch.grid(row=0, column=2, sticky="ew", padx=3, pady=5)

# Create buttons (like, dislike, get lyrics, start over, view playlist)
b_like = Button(frame1b, image = tu_photoimage, compound = RIGHT, command=like)
b_like.grid(row=1, column=0, padx=3, pady=5)

b_dislike = Button(frame1b, image = td_photoimage, compound = RIGHT, command=dislike)
b_dislike.grid(row=1, column=1, padx=3, pady=5)

b_neutral = Button(frame1b, image = r_photoimage, compound = RIGHT, command=neutral)
b_neutral.grid(row=1, column=2, padx=2, pady=5)

b_listen = Button(frame1b, image = p_photoimage, compound = RIGHT, command=listen_to_track)
b_listen.grid(row=1, column=3, padx=2, pady=5)

# Track number of lyrics liked or disliked 
tracker_label = Label(frame1, text = 'Tracks Rated: ' + str("{0:0=2d}".format(like_dislike_counter)) + "/10")
tracker_label.grid(row=2, column=0, sticky="nw", padx=5, pady = 10)

# When play is clicked, display message on where song will play 
sample_song_label = Label(frame1, text = '')
sample_song_label.grid(row=1, column=1, sticky="nw", padx=5, pady = 10, columnspan=3)

# Start over button to get back to original screen
b_startover = Button(frame3, text = '        Start Over        ', command=start_over)
b_startover.grid(row=1, column=0, sticky="ew", padx=3, pady=5)

# View playlist once 10 lyrics rated
b_playlist = Button(frame3, text = '       View Playlist       ', command=view_playlist)
b_playlist.grid(row=1, column=1, sticky="ew", padx=3, pady=5)

# Export playlist to Spotify
b_export_playlist = Button(frame3, text = 'Add to Spotify Queue', command=export_to_spotify)
b_export_playlist.grid(row=1, column=2, sticky="ew", padx=3, pady=5)

# Store selection for likes and dislikes
#data = []    

# Set original state for like and dislike buttons to disables
b_like["state"] = DISABLED
b_dislike["state"] = DISABLED
b_neutral["state"] = DISABLED
b_listen["state"] = DISABLED
b_playlist["state"] = DISABLED
b_export_playlist["state"] = DISABLED

# create table to show lyrics with rating and add a scrollbar
cols = ('Track Name', "Artist Name", 'Rating')
listBox_lyrics = ttk.Treeview(frame2, columns=cols, show='headings')
vsb_lyrics = ttk.Scrollbar(frame2, orient="vertical", command=listBox_lyrics.yview)
#doubleclicking changes like to dislike and vice versa
listBox_lyrics.bind("<Double-1>", enable_edit)

for col in cols:
    listBox_lyrics.heading(col, text=col)    
#listBox_lyrics.grid(row=1, column=0, sticky="ew")

vsb_lyrics.grid(row=1, column=1, sticky='ns')
listBox_lyrics.grid(row=1, column=0, sticky="ew")
listBox_lyrics.configure(yscrollcommand=vsb_lyrics.set)

#listBox_lyrics.pack(side="left", fill=BOTH, padx=5, pady=5)

# create table to show playlist - do not display until enabled
cols = ("Track Name","Artist Name")
listBox_playlist = ttk.Treeview(frame2, selectmode='browse', columns=cols, show='headings')
vsb_playlist = ttk.Scrollbar(frame2, orient="vertical", command=listBox_playlist.yview)

for col in cols:
    listBox_playlist.heading(col, text=col)

mainloop()


test


In [11]:
#data

In [12]:
#track = '5jzEwSyyymBlf1fa1o39T2'
#track = sp.track(track)
#print(track)
#print(track["preview_url"])

In [13]:
#sp.add_to_queue('69lfFTtbMuU99iZb67X9J7')

In [48]:
#mainloop()